In [ ]:
import pandas as pd
import datetime
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

%run Constant.ipynb
%run Account.ipynb


class AccountSavior:
    basic_info_file = 'basic_info'
    account_deals_file='account_deals'
    account_position_file ='position'
    account_increase_per_file = 'increase_per'
    account_asset_file = 'asset'
    
    def __init__(self,is_on_prod):
        self.is_on_prod = is_on_prod
    
     
    def build_persist_file_path(self,type_name,cur_date):
        account_data_path = ''
        if(self.is_on_prod==True):
            account_data_path = account_prod_data_dir
        else:
            account_data_path = account_draft_data_dir
        
        return account_data_path+'account_data_'+type_name+'_'+account_data_version+'_'+cur_date+'.csv'
        
    
    def persist_account(self, account,cur_date):
        basic_info_df = pd.DataFrame(columns = account_basic_info_columns)
        basic_info_df.loc[basic_info_df.shape[0]+1] = [account.context['short'],account.context['mid'],account.context['level'],
                                                       account.policy_name,account.init_money,
                                                       account.account_money,account.honor_account['total_asset']]
        basic_info_df.to_csv(self.build_persist_file_path(self.basic_info_file,cur_date),index=False)
        
            
        account.deal_df.to_csv(self.build_persist_file_path(self.account_deals_file,cur_date),index=False)
        account.position_df.to_csv(self.build_persist_file_path(self.account_position_file,cur_date),index=False)
        
        increase_per_file_path = self.build_persist_file_path(self.account_increase_per_file,cur_date)
        account.cur_increase_per_df.to_csv(increase_per_file_path,index=False)
        
        account.asset.to_csv(self.build_persist_file_path(self.account_asset_file,cur_date),index=False)
        
        
    def did_persist(self,cur_date):
        file = self.build_persist_file_path(self.account_deals_file,cur_date)
        if(os.path.exists(file)):
            return True
        else:
            return False
        
    def restore_account(self, cur_date):
        if(self.did_persist(cur_date)==False):
            print('Warning! can not found persisted account files '+cur_date)
            return None
        
        basic_info_df = pd.read_csv(self.build_persist_file_path(self.basic_info_file,cur_date))
        
        policy_name =basic_info_df['policy_name'].values[0]
        self.policy_loader = Policy_Loader(policy_name)
        context = self.policy_loader.get_context()
        
        account = Account(context,policy_name)
        
        account.deal_df = pd.read_csv(self.build_persist_file_path(self.account_deals_file,cur_date))
        account.position_df = pd.read_csv(self.build_persist_file_path(self.account_position_file,cur_date))
        
        account.cur_increase_per_df = pd.read_csv(self.build_persist_file_path(self.account_increase_per_file,cur_date))
        
        account.asset = pd.read_csv(self.build_persist_file_path(self.account_asset_file,cur_date))
        
        account.account_money = basic_info_df['account_money'].values[0]
        
        honor_account_total_asset = basic_info_df['honor_account_total_asset'].values[0]
        account.honor_account['total_asset'] = honor_account_total_asset
            
        symbols = account.get_all_position_symbols()
        account.metric_d_k = {}
        for symbol in symbols:
            account.metric_d_k[symbol] = Metric(symbol,context)
        
        return account
    
    
    def to_str(self,account):
        print('------------------------account desc start------------------------------------')
        if(account.asset.shape[0]>0):
            display(account.deal_df[:3])
            display(account.position_df[:3])
            display(account.cur_increase_per_df[:3])
            display(account.asset[:3])
        print('------------------------account desc end  ------------------------------------')
        
        
    
        
    